In [1]:
import pennylane as qml
from pennylane import numpy as np

In [119]:
T = qml.T.compute_matrix()
X = qml.PauliX.compute_matrix()
I = np.eye(2)

U = np.kron(T, np.kron(X, I))

U

eigvals, eigvecs = np.linalg.eig(U)

target_eigvec = eigvecs[:, 7]
target_eigval = eigvals[7]
np.angle(target_eigval) / (2 * np.pi)  # 001
target_eigvec

tensor([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  7.07106781e-01+0.00000000e+00j,
        -2.16432722e-17-1.14967359e-17j, -7.07106781e-01-1.38777878e-17j], requires_grad=True)

In [120]:
np.angle(target_eigval) / (2 * np.pi) # 001

-0.375

In [121]:
U11 = [0.58939515 - 0.13274642j, 0.64381996 + 0.46956108j, 0.79653634 + 0.02283875j, -0.38478345 - 0.46577886j]
U12 = [-0.25432824 + 0.63921011j, -0.62853537 - 0.3628648j, 0.56165482 + 0.45964273j, 0.50800259 - 0.46390278j]
U13 = [0.18203758 + 0.17691194j, -0.73088329 - 0.63354092j, 0.96399606 - 0.07922173j, 0.25139533 - 0.03515543j]

In [122]:
U11 = np.array(U11).reshape((2, 2))
U12 = np.array(U12).reshape((2, 2))
U13 = np.array(U13).reshape((2, 2))

In [123]:
U21 = [-0.38244183 - 0.12961519j, -0.58523477 - 0.07874502j, 0.30743697 - 0.1900339j,
       0.35244239 - 0.48310622j, 0.0520515 + 0.11832368j, -0.31650125 - 0.29741055j,
       -0.43985789 + 0.65802989j, -0.22420903 - 0.3433891j, 0.80468183 - 0.37828889j,
       -0.17001017 - 0.01590874j, -0.00860685 - 0.28232261j, -0.10595894 - 0.29869833j,
       0.17166715 + 0.01197327j, -0.3397648 + 0.56398292j, -0.37158034 + 0.15823791j,
       0.54409558 + 0.27878125j]
U22 = [0.39021299 + 0.19550829j, 0.51227196 + 0.31891725j, 0.29405574 - 0.23122613j,
       0.05455699 + 0.54997163j, -0.01534597 + 0.4880493j, 0.32273224 - 0.0277232j,
       -0.78572909 - 0.00924454j, -0.19764638 - 0.01131186j, 0.61343005 + 0.33688749j,
       -0.55122531 + 0.10760665j, -0.08187687 - 0.30746768j, 0.22104702 - 0.21137614j,
       0.1761403 + 0.22420457j, 0.45391289 - 0.09736075j, 0.37273568 + 0.04937567j,
       -0.14723682 - 0.73494378j]
U23 = [0.09993947 + 0.22310773j, -0.52519719 - 0.19872825j, 0.49386529 + 0.09293278j,
       -0.45451301 + 0.40717139j, 0.15789114 + 0.17890024j, 0.01504621 - 0.04207803j,
       -0.6909489 + 0.35722228j, -0.03586294 + 0.5785878j, 0.40172155 - 0.35194883j,
       -0.31727546 - 0.66402832j, -0.20322529 - 0.28188923j, 0.2074951 - 0.0966345j,
       -0.75580237 + 0.16375204j, -0.07412908 - 0.36821951j, 0.02474916 - 0.14514768j,
       0.40442414 + 0.27500556j]

In [124]:
U21 = np.array(U21).reshape((4, 4))
U22 = np.array(U22).reshape((4, 4))
U23 = np.array(U23).reshape((4, 4))

In [125]:
eigvals, eigvecs = np.linalg.eig(U11)

In [126]:
eigvals[0]

(0.9999710236265978+0.007612942034758319j)

In [127]:
np.angle(eigvals[0]) / (2 * np.pi)

0.001211649057380608

In [128]:
eigvecs[0]

tensor([ 0.878242  +0.j        , -0.45675296-0.14166061j], requires_grad=True)

In [132]:
import pennylane as qml
from pennylane import numpy as np


def construct_estimation_qnode(U, eigvec, n_estimation_wires):
    """Construct and return a QNode that will perform QPE with a given number
    of estimation wires.

    Use the QuantumPhaseEstimation template.

    Args:
        U (array[complex]): A random 1-qubit or 2-qubit unitary matrix.
        eigvec (array[complex]): An eigenvector whose eigenvalue we are trying to find.
        n_estimation_wires (int): The number of estimation wires to use.

    Returns:
        QNode: A QNode that performs QPE for U and the given eigenvector
        and uses the provided number of estimation wires.
    """

    n_trg_wires = int(np.log2(len(U)))

    target_wires = np.arange(0, n_trg_wires).tolist()
    estimation_wires = np.arange(n_trg_wires, n_estimation_wires + n_trg_wires).tolist()

    dev = qml.device("default.qubit", wires=estimation_wires + target_wires)

    @qml.qnode(dev)
    def qnode():
        """Your QNode should not take any parameters as input."""
        qml.MottonenStatePreparation(eigvec, wires=target_wires)
        qml.QuantumPhaseEstimation(
            U,
            target_wires=target_wires,
            estimation_wires=estimation_wires
        )
        # return qml.sample(wires=n_estimation_wires)
        return qml.probs(wires=estimation_wires)

    return qnode


# From L11 Demo
def fractional_binary_to_float(sample):
    """Convert an n-bit sample [k1, k2, ..., kn] to a floating point
    value using fractional binary representation,

        k = (k1 / 2) + (k2 / 2 ** 2) + ... + (kn / 2 ** n)

    Args:
        sample (list[int] or array[int]): A list or array of bits, e.g.,
            the sample output of quantum circuit.

    Returns:
        float: The floating point value corresponding computed from the
        fractional binary representation.
    """
    return np.sum(
        [int(sample[bit]) / 2 ** (bit + 1) for bit in range(len(sample))]
    )


def dec_to_bin(x):
    if x == 0:
        return [0]
    bit = []
    while x:
        bit.append(x % 2)
        x >>= 1
    return bit[::-1]


def compute_estimation_window(U, eigvec):
    """Use QPE to compute a phase window that bounds the value of a phase
    that cannot be expressed exactly using t bits.

    Suppose the true value of the phase is exp(2 * np.pi * 1j * theta).  The
    phase window should be an array of two real numbers [theta_lower,
    theta_upper], such that the interval between them contains the true value of
    theta. The width of the window should be no larger than 1e-3; you will need
    to determine the number of estimation wires required to obtain a window to
    this precision.

    Your code will be tested on 1-qubit unitaries, and 2-qubit unitaries.

    Args:
        U (array[complex]): A random 1-qubit or 2-qubit unitary matrix.
        eigvec (array[complex]): An eigenvector whose eigenvalue we are trying to find.

    Returns:
        array[float]: An array representing the lower and upper bound of the
        value of the phase we are estimating (the absolute difference between
        the two values should be less than 1e-3).
    """

    # Need 10 wires for window width < 0.001 where 1/(2^10) = 0.000976

    phase_window = [0.0, 1.0]

    n_est_wires = 3

    target = construct_estimation_qnode(U, eigvec, n_est_wires)()

    # eigen_value = (1 / (2 ** (np.argmax(target) + 1)))

    theta = np.argmax(target) / (2 ** n_est_wires)

    # return [eigen_value - (1 / (2 ** n_est_wires)), eigen_value + (1 / (2 ** n_est_wires))], eigen_value
    return [theta - (1 / (2 ** n_est_wires)), theta + (1 / (2 ** n_est_wires))], theta


# compute_estimation_window(U11, eigvecs[0])
compute_estimation_window(U, target_eigvec)

([0.5, 0.75], 0.625)

In [133]:
construct_estimation_qnode(U, target_eigvec, 3)()

tensor([5.84611416e-33, 5.91857596e-32, 5.19858605e-33, 2.36034631e-32,
        2.11816813e-31, 1.00000000e+00, 6.53859303e-31, 2.35389673e-31], requires_grad=True)

In [131]:
construct_estimation_qnode(U, target_eigvec, 3)()

tensor([5.84611416e-33, 5.91857596e-32, 5.19858605e-33, 2.36034631e-32,
        2.11816813e-31, 1.00000000e+00, 6.53859303e-31, 2.35389673e-31], requires_grad=True)

In [33]:
1 / (2 ** 3)

0.125